In [1]:
import sys
from pathlib import Path

In [2]:
module_path = Path.cwd().parent.parent
if module_path not in sys.path:
    sys.path.append(str(module_path))

In [3]:
from flair.datasets import ColumnCorpus
from flair.data import Corpus, Sentence
from flair.embeddings import WordEmbeddings, StackedEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

/home/marek/anaconda3/envs/nlp/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from src.loader import TextLoader

In [5]:
loader = TextLoader()

In [6]:
columns = {0: "text", 1: "ner"}
corpus: Corpus = ColumnCorpus(
    "data/",
    columns,
    train_file="NER_train.txt",
    dev_file="NER_dev.txt",
    test_file="NER_test.txt",
    column_delimiter=" ",
    document_separator_token="<DOC>",
)
corpus.filter_empty_sentences()


2022-12-03 15:15:09,878 Reading data from data
2022-12-03 15:15:09,878 Train: data/NER_train.txt
2022-12-03 15:15:09,879 Dev: data/NER_dev.txt
2022-12-03 15:15:09,879 Test: data/NER_test.txt
2022-12-03 15:15:47,173 Filtering empty sentences
2022-12-03 15:15:49,734 Corpus: 250135 train + 46666 dev + 15696 test sentences


In [7]:
label_dict = corpus.make_label_dictionary(label_type='ner')

2022-12-03 15:15:49,755 Computing label dictionary. Progress:


250135it [00:02, 94395.77it/s]

2022-12-03 15:15:52,408 Dictionary created for label 'ner' with 3 values: DE (seen 115092 times), DS (seen 114349 times)


In [8]:
embedding_types = [
    WordEmbeddings('glove'),
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

In [9]:
tagger = SequenceTagger(
    hidden_size=256,
    embeddings=embeddings,
    tag_dictionary=label_dict,
    tag_type="ner",
    use_crf=True,
)


2022-12-03 15:15:54,240 SequenceTagger predicts: Dictionary with 3 tags: <unk>, DE, DS


In [10]:
trainer = ModelTrainer(tagger, corpus)

In [11]:
trainer.train(
    'models/',
    learning_rate=0.1,
    mini_batch_size=32,
    max_epochs=20,
    patience=4,
    embeddings_storage_mode='gpu',
    checkpoint=True,
)


2022-12-03 15:15:55,935 ----------------------------------------------------------------------------------------------------
2022-12-03 15:15:55,936 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'glove'
      (embedding): Embedding(400001, 100)
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=100, out_features=100, bias=True)
  (rnn): LSTM(100, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=5, bias=True)
  (loss_function): ViterbiLoss()
  (crf): CRF()
)"
2022-12-03 15:15:55,936 ----------------------------------------------------------------------------------------------------
2022-12-03 15:15:55,936 Corpus: "Corpus: 250135 train + 46666 dev + 15696 test sentences"
2022-12-03 15:15:55,937 ----------------------------------------------------------------------------------------------------
2022-12-03 15:15:55,

100%|██████████| 1459/1459 [01:43<00:00, 14.06it/s]


2022-12-03 15:23:18,384 Evaluating as a multi-label problem: False
2022-12-03 15:23:21,337 DEV : loss 0.10346631705760956 - f1-score (micro avg)  0.0017
2022-12-03 15:23:21,886 BAD EPOCHS (no improvement): 0
2022-12-03 15:23:22,593 saving best model
2022-12-03 15:23:23,188 ----------------------------------------------------------------------------------------------------
2022-12-03 15:23:56,567 epoch 2 - iter 781/7817 - loss 0.10559244 - samples/sec: 750.63 - lr: 0.100000
2022-12-03 15:24:28,921 epoch 2 - iter 1562/7817 - loss 0.10599293 - samples/sec: 773.97 - lr: 0.100000
2022-12-03 15:25:01,290 epoch 2 - iter 2343/7817 - loss 0.10602334 - samples/sec: 773.57 - lr: 0.100000
2022-12-03 15:25:33,560 epoch 2 - iter 3124/7817 - loss 0.10576322 - samples/sec: 775.94 - lr: 0.100000
2022-12-03 15:26:06,131 epoch 2 - iter 3905/7817 - loss 0.10575062 - samples/sec: 768.76 - lr: 0.100000
2022-12-03 15:26:38,588 epoch 2 - iter 4686/7817 - loss 0.10542264 - samples/sec: 771.47 - lr: 0.100000
20

100%|██████████| 1459/1459 [01:42<00:00, 14.17it/s]


2022-12-03 15:30:34,438 Evaluating as a multi-label problem: False
2022-12-03 15:30:37,426 DEV : loss 0.096551813185215 - f1-score (micro avg)  0.0213
2022-12-03 15:30:37,981 BAD EPOCHS (no improvement): 0
2022-12-03 15:30:38,339 saving best model
2022-12-03 15:30:38,706 ----------------------------------------------------------------------------------------------------
2022-12-03 15:31:12,050 epoch 3 - iter 781/7817 - loss 0.10363700 - samples/sec: 751.01 - lr: 0.100000
2022-12-03 15:31:45,454 epoch 3 - iter 1562/7817 - loss 0.10369032 - samples/sec: 749.54 - lr: 0.100000
2022-12-03 15:32:18,197 epoch 3 - iter 2343/7817 - loss 0.10351495 - samples/sec: 764.70 - lr: 0.100000
2022-12-03 15:32:50,718 epoch 3 - iter 3124/7817 - loss 0.10308222 - samples/sec: 769.90 - lr: 0.100000
2022-12-03 15:33:23,985 epoch 3 - iter 3905/7817 - loss 0.10296485 - samples/sec: 752.63 - lr: 0.100000
2022-12-03 15:33:57,178 epoch 3 - iter 4686/7817 - loss 0.10311434 - samples/sec: 754.31 - lr: 0.100000
2022

...

100%|██████████| 1459/1459 [01:38<00:00, 14.76it/s]


2022-12-03 17:05:30,232 Evaluating as a multi-label problem: False
2022-12-03 17:05:33,387 DEV : loss 0.0882866233587265 - f1-score (micro avg)  0.0186
2022-12-03 17:05:33,956 BAD EPOCHS (no improvement): 1
2022-12-03 17:05:34,666 ----------------------------------------------------------------------------------------------------
2022-12-03 17:05:44,566 ----------------------------------------------------------------------------------------------------
2022-12-03 17:05:44,567 Exiting from training early.
2022-12-03 17:05:44,568 Saving model ...
2022-12-03 17:05:44,928 Done.
2022-12-03 17:05:44,930 ----------------------------------------------------------------------------------------------------
2022-12-03 17:05:44,932 loading file models/best-model.pt
2022-12-03 17:05:45,340 SequenceTagger predicts: Dictionary with 5 tags: <unk>, DE, DS, <START>, <STOP>


100%|██████████| 491/491 [00:35<00:00, 13.71it/s]


2022-12-03 17:06:21,583 Evaluating as a multi-label problem: False
2022-12-03 17:06:22,592 0.0159	0.3609	0.0305	0.0159
2022-12-03 17:06:22,593 
Results:
- F-score (micro) 0.0305
- F-score (macro) 0.2834
- Accuracy 0.0159

By class:
              precision    recall  f1-score   support

       <unk>     0.0000    0.0000    0.0000         0
          DE     0.5210    0.4189    0.4644      7302
          DS     0.5322    0.3024    0.3856      7243

   micro avg     0.0159    0.3609    0.0305     14545
   macro avg     0.3511    0.2404    0.2834     14545
weighted avg     0.5266    0.3609    0.4252     14545

2022-12-03 17:06:22,594 ----------------------------------------------------------------------------------------------------


{'test_score': 0.030521966919514233,
 'dev_score_history': [0.0016527694180800817,
  0.02129778670059778,
  0.01713351357604014,
  0.012251667963917476,
  0.020119477779760146,
  0.0030741903292402944,
  0.013012373223959078,
  0.02248197736313677,
  0.014241657239541775,
  0.020433170670498952,
  0.013316263211862297,
  0.015400360354708235,
  0.022407475301586304,
  0.03076935142034259,
  0.018611791323646758],
 'train_loss_history': [0.11162724334505603,
  0.10472340281774903,
  0.10285513800329801,
  0.10183264383369618,
  0.10100515115303266,
  0.10053029851898,
  0.10019411184330691,
  0.09662685535563979,
  0.09592075308415242,
  0.09555069143122688,
  0.09527929121687452,
  0.09511848705341996,
  0.09487610242898997,
  0.09324911343296113,
  0.09291608804392594],
 'dev_loss_history': [0.10346631705760956,
  0.096551813185215,
  0.09422393143177032,
  0.0936627984046936,
  0.09435790777206421,
  0.09946397691965103,
  0.09262198954820633,
  0.09051612764596939,
  0.0894617363810

In [12]:
model = SequenceTagger.load("models/final-model.pt")

2022-12-03 17:10:16,747 loading file models/final-model.pt
2022-12-03 17:10:17,370 SequenceTagger predicts: Dictionary with 5 tags: <unk>, DE, DS, <START>, <STOP>


In [13]:
text = loader.load_random_text()

sent = Sentence(text.text)
model.predict(sent)

In [15]:
for disc in text.discourses:
    print(disc)

--- 1621354891977 (0 -> 40 | 0 -> 6) - Position ---
The driverless Cars article I am against
---------------------------------------------------
--- 1621354905027 (49 -> 91 | 8 -> 15) - Claim ---
What benefits would we get from these cars
--------------------------------------------------
--- 1621354913903 (92 -> 152 | 16 -> 27) - Claim ---
Will auto body parts run out of buisness because of the cars
----------------------------------------------------
--- 1621354921464 (153 -> 192 | 28 -> 34) - Claim ---
What will happen to taxi drivers jobs. 
-----------------------------------------------------
--- 1621354933349 (193 -> 238 | 35 -> 43) - Claim ---
What benefits would we get out of these cars?
-----------------------------------------------------
--- 1621354948467 (239 -> 278 | 44 -> 51) - Counterclaim ---
Maybe we will get less wrecks with them
------------------------------------------------------------
--- 1621354967258 (279 -> 526 | 52 -> 96) - Rebuttal ---
but think of this what

In [14]:
sent.to_tagged_string()

'Sentence: "The driverless Cars article I am against because What benefits would we get from these cars Will auto body parts run out of buisness because of the cars What will happen to taxi drivers jobs . What benefits would we get out of these cars ? Maybe we will get less wrecks with them but think of this what will happen if your car sensors and autopilot does n\'t work anymore the passangers will have to drive but what if they do n\'t know how to drive because the driving classes got taken away because these cars do n\'t need a driver . It also shows in the text that if something bad happens who \'s fault is it the manufacturer or the driver in this case it would be the manufacturers fault because they did n\'t check the car throughly and shipped it somwhere to be sold to some people then thoese people get stuck in the middle of nowhere because of the driverless car and the manufacturer but what will happen if the car gets struck by lightning because of all the electrical systems t